# Make sure to run this in the `my-torch` environment!

Link to tutorial: https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

> Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). 

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
torch.cuda.is_available()

False

In [3]:
# GPU not available, therfore use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# create a neural network by sub-classing nn.Module 
# define initialization and `forward` method
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # the following stack combines two linear layers with ReLU activation
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=28 * 28, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=10)
        )
        self.flatten = nn.Flatten() ##
    
    def forward(self, x):
        x = self.flatten() ##
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
28 * 28 * 512

401408